## Objective
Extract the spike trough amplitudes of all threshold crossings across all electrodes
for "Follow up propagation signal algorithm test"

In [38]:
# %load_ext autoreload
# %autoreload

from scipy.io import savemat, loadmat
from multiprocessing import Pool
import numpy as np
from spikeinterface.extractors import NwbRecordingExtractor
from spikeinterface.preprocessing import bandpass_filter, scale
from tqdm import tqdm

In [26]:
# Get recording
REC_PATH = "/data/MEAprojects/dandi/000034/sub-mouse412804/sub-mouse412804_ecephys.nwb"
# Thresh crossing path
THRESH_CROSSINGS_PATH = "/data/MEAprojects/dandi/000034/sub-mouse412804/prop_signal/thresh_crossings_5_neg_only.npy"
##
rec = NwbRecordingExtractor(REC_PATH)
rec = scale(rec, dtype="float32")
rec = bandpass_filter(rec, freq_min=300, freq_max=3000, dtype="float32")  # type: NwbRecordingExtractor

In [ ]:
# Extract amplitudes
def get_amplitudes(idx):
    extract_times = np.load(THRESH_CROSSINGS_PATH, allow_pickle=True)[idx]

    amps = []
    for t in extract_times:
        time = int(t * rec.get_sampling_frequency() / 1000)
        amps.append(rec.get_traces(start_frame=time, end_frame=time+1).flatten())
    return amps

amps_all = []
num_crossings = len(np.load(THRESH_CROSSINGS_PATH, allow_pickle=True))
with Pool(processes=8) as pool:
    tasks = range(num_crossings)
    for amps in tqdm(pool.imap(get_amplitudes, tasks, chunksize=1), total=len(tasks)):
        amps_all.append(amps)

 37%|███▋      | 91/248 [18:44<53:18, 20.37s/it]  

In [37]:
np.save("/data/MEAprojects/dandi/000034/sub-mouse412804/prop_signal/for_tj_email_follow_up_propagation_signal_alg_test/amps_thresh_crossings.npy", amps_all)

/home/mea/anaconda3/envs/allen_inst/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [41]:
# Format data
amps_formatted = {}
for i in range(len(amps_all)):
    amps = np.vstack([amps_all[i]])
    amps_formatted[f"c{i}"] = amps

In [42]:
print(amps_formatted.keys())

dict_keys(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75', 'c76', 'c77', 'c78', 'c79', 'c80', 'c81', 'c82', 'c83', 'c84', 'c85', 'c86', 'c87', 'c88', 'c89', 'c90', 'c91', 'c92', 'c93', 'c94', 'c95', 'c96', 'c97', 'c98', 'c99', 'c100', 'c101', 'c102', 'c103', 'c104', 'c105', 'c106', 'c107', 'c108', 'c109', 'c110', 'c111', 'c112', 'c113', 'c114', 'c115', 'c116', 'c117', 'c118', 'c119', 'c120', 'c121', 'c122', 'c123', 'c124', 'c125', 'c126', 'c127', 'c128', 'c129', 'c130', 'c131', 'c132', 'c133', 'c134', 'c135', 'c136', 'c1

In [44]:
amps_formatted["c206"].shape

(14923, 248)

In [45]:
savemat("/data/MEAprojects/dandi/000034/sub-mouse412804/prop_signal/for_tj_email_follow_up_propagation_signal_alg_test/230130_thresh_crossings_amps.mat", amps_formatted)